In [1]:
import pickle



In [8]:
rez = pickle.load(open("./checkpoint_4", "rb"))

In [10]:
print(rez.keys())

dict_keys(['last_iteration', 'elements'])


In [12]:
len(rez["elements"])

48

In [15]:
rez["elements"][0].result_elements

[ResultItem(result_id='121780', lot_id='1 ', price=None, currency='euro'),
 ResultItem(result_id='121780', lot_id='2 ', price='800', currency='euro'),
 ResultItem(result_id='121780', lot_id='3 ', price=None, currency='euro'),
 ResultItem(result_id='121780', lot_id='4 ', price='400', currency='euro'),
 ResultItem(result_id='121780', lot_id='5 ', price='620', currency='euro'),
 ResultItem(result_id='121780', lot_id='6 ', price='150', currency='euro'),
 ResultItem(result_id='121780', lot_id='7 ', price='220', currency='euro'),
 ResultItem(result_id='121780', lot_id='8 ', price='200', currency='euro'),
 ResultItem(result_id='121780', lot_id='9 ', price='750', currency='euro'),
 ResultItem(result_id='121780', lot_id='10 ', price='420', currency='euro'),
 ResultItem(result_id='121780', lot_id='12 ', price='150', currency='euro'),
 ResultItem(result_id='121780', lot_id='13 ', price='570', currency='euro'),
 ResultItem(result_id='121780', lot_id='14 ', price='370', currency='euro'),
 ResultIte

In [17]:
rez["elements"][0].catalog_elements[0]

CatalogItem(catalog_id='121780-tableaux-mobilier-et-objets-d-art-argenterie-art-d-asie', lot_id='/lots/17058994-attribue-a-adriaen-van-der', presentation="\n\nLot n° 1\n\n                                    \n                                        Attribué à Adriaen VAN DER KABEL…\n                                    \n                                \n\n                                    Attribué à Adriaen VAN DER KABEL (1631-1705)\nVue d'un port au Levant\nHuile sur toile\n40 x 54 cm\n(cadre en bois doré )\n                                \n\n", result=None, estimation='\n                                    Résultat :\n                                    \n                                        \n                                            3\xa0100 EUR\n                                        \n                                    \n                                    \n                                        /\n                                        Estimation :\n                

In [50]:
rez["elements"][0].catalog_elements[11]

CatalogItem(catalog_id='121780-tableaux-mobilier-et-objets-d-art-argenterie-art-d-asie', lot_id='/lots/17059004-hilaire-le-dru-1769-1840--les', presentation='\n\nLot n° 11\n\n                                    \n                                        Hilaire LE DRU (1769-1840)\nLes…\n                                    \n                                \n\n                                    Hilaire LE DRU (1769-1840)\nLes pénibles adieux\nReprésentant du peuple entouré de sa famille déposant des fleurs sur la tombe de son épouse\ndessin au crayon sur papier\nSigné en bas à gauche\n31, 5 x 44 cm\n\nCe dessin fut exposé au salon de 1796 \nVente Pierre…\n                                \n\n', result=None, estimation='\n                                    Résultat :\n                                    \n                                        \n                                            Non Communiqué\n                                        \n                                    \n   

In [305]:
from collections import Counter
import itertools


def parse_to_text_tokens(presentation):
    return list(
        map(lambda x: x.strip(" …"),
            filter(lambda x: len(set(x)) > 1, 
                presentation.split("\n"))
    ))



def identify_lot_infos(lot_tokens):
    funcs = [identify_lot_id, identify_date, identify_size, identify_volume, identify_author, 
             identify_fake, identify_authenticity]
    identified_tokens = list(
        filter(None, map(lambda x: x[0](x[1]), itertools.product(funcs, lot_tokens)))
    )
    return identified_tokens
    

def identify_estimation_and_price(filtered_str):
    match = re.search("\s+Résultat\s:\s+(?P<result>[\w]*)\s(?P<currency_opt>[\w]*).*Estimation :\s+(?P<estimation_low>[0-9]+)\s- (?P<estimation_high>[0-9]+) (?P<currency>[A-Z]*)\s+.", 
              filtered_str)
    if match:
        return match.groupdict()
    
def str_reverse(str_):
    return "".join(reversed(str_))

def identify_author(filtered_str):
    """
    Pattern 1, 2 : match "Jean-Sébastien D'ASNIERE POUET"
    Pattern 3 : match "D'après Frank RIBERA"
    Pattern 4 : match "D'après RIBERA"
    """
    match = re.search("\)\s*(?P<death>[0-9]{4})\s*\-\s*(?P<birth>[0-9]{4})\s*\(\s+(?P<surname>[A-Z -']+)\s+(?P<name>([A-Za-z -'éèê]+[A-Z])+)", str_reverse(filtered_str))
    if match:
        return {k: str_reverse(v) for k, v in match.groupdict().items()}
    match = re.search("(?P<name>[A-Za-z -'éèê]+)\s+(?P<surname>[A-Z -']+)\s+\(\s*(?P<birth>[0-9]{4})\s*-\s*(?P<death>[0-9]{4})\s*\)", filtered_str)
    if match:
        return match.groupdict()
    match = re.search("d?D?'après (?P<name>[A-Za-z -'éèê]+)\s+(?P<surname>[A-Z -']+)", filtered_str)
    if match:
        return match.groupdict()
    match = re.search("d?D?'après (?P<surname>[A-Z -']+)", filtered_str)
    if match:
        return match.groupdict()
    
def identify_volume(filtered_str):
    match = re.search("H\s?:\s?(?P<height>[0-9]+)\W*L\s?:\s?(?P<width>[0-9]+)\W*P\s?:\s?(?P<depth>[0-9]+)\s?(?P<unit>[a-z]{0,3})", filtered_str)
    if match:
        return match.groupdict()
    
def identify_date(filtered_str):
    for match_format in ["(1[0-9]{3})", "(20[0-2]{1}[0-9]{1})"]:
        date_format_one = re.search(match_format, filtered_str)
        if date_format_one is not None:
            return int(date_format_one[1])
    for match_format in ["(1[0-9]{1})\s?ème", "(2[0-1]{1})\s?ème"]:
        date_format_one = re.search(match_format, filtered_str)
        if date_format_one is not None:
            return (int(date_format_one[1]) - 1) * 100 + 50

def identify_lot_id(filtered_str):
    match = re.match("Lot n° (?P<lot_id>[0-9]+)\s?(?P<bis>b?i?s?t?e?r?)", filtered_str)
    if match:
        return match.groupdict()
    
def identify_fake(filtered_str):
    keyword_list = ["Dans le goût de", "D'après"]
    if any(k.lower() in filtered_str.lower() for k in keyword_list):
        return {"fake": True}

def identify_authenticity(filtered_str):
    keyword_list = ["signé", "signature", "initiales"]
    if any(k.lower() in filtered_str.lower() for k in keyword_list):
        return {"authentic": True}


In [ ]:

all_items = list()
for sale in rez["elements"]:
    if sale is None or sale.catalog_elements is None:
        continue
    for element in sale.catalog_elements:
        current_item = list()
        current_item.extend(identify_lot_infos(parse_to_text_tokens(element.presentation)))
        current_item.append(
            identify_estimation_and_price(element.estimation.replace("\n", " ").replace("\xa0", ""))
        )
        all_items.append((current_item, element.presentation))

for i, (x, y) in enumerate(all_items):
    print("--------")
    print(y)
    print(x)
    print("\n\n\n")
    if i > 40:
        break
# TODO : remove substrings, remove superfluous spaces, parse Lot id, try to parse size
# words = [Aquarelle, Paysage de forêt, Huile sur toile, Gouache sur papier, Pastel sur papier]

In [187]:
import re



identify_estimation_and_price(rez["elements"][0].catalog_elements[11].estimation.replace("\n", " ").replace("\xa0", ""))

{'result': 'Non Communiqué',
 'estimation_low': '200',
 'estimation_high': '400',
 'currency': 'EUR'}

In [55]:
group.groupdict()

{'result': 'Non Communiqué', 'estimation': '200 - 400 EUR'}

In [56]:
group[2]

'200 - 400 EUR'

In [100]:
identify_lot_id("Lot n° 3")

('3', '')

In [121]:
identify_date("19 ème siècle")

1850

In [122]:
identify_date("21 ème siècle")

2050

In [123]:
identify_date("1834")

1834

In [124]:
identify_date("2013")

2013

In [134]:
def identify_size(filtered_str):
    found = re.search("([0-9]+)\s?x\s?([0-9]+)\s?([a-z]{0,3})", filtered_str)
    if found:
        return {"height": found[1], "width": found[2], "unit": found[3]}

In [135]:
identify_size("70 x 90")

('70', '90', '')

In [159]:
identify_volume('H : 80 - L: 127- P: 68')

{'height': '80', 'width': '127', 'depth': '68', 'unit': ''}

In [160]:
identify_volume('H : 80 - L: 127- P: 68')

{'height': '80', 'width': '127', 'depth': '68', 'unit': ''}

In [291]:
a = "(?P<name>[\b[A-Za-z]+\s?-?]+)\s+(?P<surname>[A-Z\s-]+)\s+\(\s*(?P<birth>[0-9]{4})\s*-\s*(?P<death>[0-9]{4})\s*\)"

b = "\)\s*(?P<death>[0-9]{4})\s*\-\s*(?P<birth>[0-9]{4})\s*\(\s+(?P<surname>[A-Z -']+)\s+(?P<name>([A-Za-z -'éèê]*[A-Z])+)"

In [293]:
p = re.compile(b)
p.search(str_reverse("tableau de Jean Sébastien D'ASNIERE POUET  (  1907 - 1982)  ")).groupdict()

{'death': '2891',
 'birth': '7091',
 'surname': "TEUOP EREINSA'D",
 'name': 'neitsabéS naeJ'}

In [271]:
identify_author("Jean-Sébastien D'ASNIERE POUET  (  1907 - 1982)  ")

{'death': '1982', 'birth': '1907', 'surname': 'POUET', 'name': "D'ASNIERE"}

In [215]:
str_reverse("abc")

'cba'

In [161]:
identify_lot()

In [164]:
dir(itertools)

['__doc__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 '_grouper',
 '_tee',
 '_tee_dataobject',
 'accumulate',
 'chain',
 'combinations',
 'combinations_with_replacement',
 'compress',
 'count',
 'cycle',
 'dropwhile',
 'filterfalse',
 'groupby',
 'islice',
 'permutations',
 'product',
 'repeat',
 'starmap',
 'takewhile',
 'tee',
 'zip_longest']

In [166]:
list(itertools.product([1, 3], [2, 4]))

[(1, 2), (1, 4), (3, 2), (3, 4)]

In [183]:
data = ['Gouache sur papier',
         'Lot n° 30',
         'Paysage de neige',
         '22 x 27 cm',
         'Lot n° 31',
         'Philippe VEYRIN (1900- 1962)',
         'Maison']
identify_lot_infos(data)

[{'lot_id': '30', 'bis': ''},
 {'lot_id': '31', 'bis': ''},
 1900,
 ('22', '27', 'cm'),
 {'name': 'Philippe', 'surname': 'VEYRIN', 'birth': '1900', 'death': '1962'}]